In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/ingredients101recipes5k"))
#print(os.listdir("../input/food41"))
from tqdm import tqdm, tqdm_notebook

# Any results you write to the current directory are saved as output.

['recipes5k', 'ingredients101']


In [2]:
print(os.listdir("../input/ingredients101recipes5k/ingredients101/Ingredients101/"))

['README.md', 'annotations', 'ingredients_simplification']


In [3]:
%matplotlib inline
import os
import sys
import cv2
import numpy as np
import torch
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn as nn
from torchvision import datasets, transforms
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.models as models

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(42) # try and make the results more reproducible
BASE_PATH = '../input/ingredients101recipes5k/ingredients101/Ingredients101/'

In [4]:
#all_img_df = pd.DataFrame({'path': glob(os.path.join(BASE_PATH, 'images', '*', '*.jpg'))})
#all_img_df['category'] = all_img_df['path'].map(lambda x: os.path.split(os.path.dirname(x))[-1])
#https://www.kaggle.com/kmader/vgg-on-food-subset-torch/data
print(os.listdir("../input/food41/images"))
epochs = 35
batch_size = 64
MICRO_DATA = True # very small subset (just 3 groups)
SAMPLE_TRAINING = False # make train set smaller for faster iteration
IMG_SIZE = (384, 384)

['fish_and_chips', 'donuts', 'bruschetta', 'cannoli', 'lobster_roll_sandwich', 'deviled_eggs', 'french_onion_soup', 'hummus', 'miso_soup', 'spaghetti_bolognese', 'mussels', 'falafel', 'french_toast', 'waffles', 'shrimp_and_grits', 'spring_rolls', 'caesar_salad', 'hamburger', 'seaweed_salad', 'tiramisu', 'chocolate_mousse', 'grilled_salmon', 'baby_back_ribs', 'pulled_pork_sandwich', 'frozen_yogurt', 'guacamole', 'baklava', 'ravioli', 'chicken_curry', 'lobster_bisque', 'ramen', 'sashimi', 'creme_brulee', 'hot_dog', 'risotto', 'pho', 'ice_cream', 'crab_cakes', 'caprese_salad', 'edamame', 'chocolate_cake', 'hot_and_sour_soup', 'carrot_cake', 'ceviche', 'sushi', 'peking_duck', 'beignets', 'gyoza', 'cheesecake', 'garlic_bread', 'onion_rings', 'prime_rib', 'scallops', 'pizza', 'beef_carpaccio', 'red_velvet_cake', 'clam_chowder', 'fried_rice', 'pork_chop', 'bibimbap', 'apple_pie', 'tuna_tartare', 'macarons', 'eggs_benedict', 'oysters', 'churros', 'spaghetti_carbonara', 'french_fries', 'breakfa

In [5]:
#Ingredients for each class
f = open(BASE_PATH + '/annotations/ingredients_simplified.txt', "r")
ingredients = f.read().split('\n')
f.close()

In [6]:
#Classes
f = open(BASE_PATH + '/annotations/classes.txt', "r")
classes = f.read().split('\n')
f.close()

In [7]:
#Base Ingredients
f = open(BASE_PATH + '/ingredients_simplification/baseIngredients.txt', "r")
base_ing = f.read().split('\n')
f.close()

In [8]:
#Construct dataframe for dictionary
base_ing = base_ing[0].split(",")

In [9]:
base_ing
ingredients
classes

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheesecake',
 'cheese_plate',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

In [10]:
new_ingredients = []
for arr in ingredients:
    arr = arr.split(",")
    new_ingredients.append(arr)
new_ingredients

[['butter', 'flour', 'sugar', 'brown sugar', 'apple', 'cinnamon', 'nut'],
 ['baby back ribs',
  'apple',
  'salt',
  'mustard',
  'brown sugar',
  'worcestershire',
  'gin',
  'chili'],
 ['nut',
  'cinnamon',
  'bread',
  'butter',
  'phyllo dough',
  'sugar',
  'honey',
  'lemon',
  'baklava'],
 ['beef', 'lemon', 'gin', 'salt', 'pepper', 'baby arugula', 'asiago'],
 ['fat',
  'steak',
  'gin',
  'shallot',
  'parsley',
  'capers',
  'worcestershire',
  'egg',
  'black pepper',
  'crostini'],
 ['beets',
  'spinach',
  'gorgonzola',
  'nut',
  'red wine',
  'dijon mustard',
  'gin',
  'salt',
  'black pepper',
  'herbs'],
 ['water',
  'sugar',
  'yeast',
  'egg',
  'salt',
  'milk',
  'butter',
  'flour',
  'sugar'],
 ['grain',
  'steak',
  'soybean sprouts',
  'spinach',
  'cucumber',
  'zucchini',
  'carrot',
  'garlic',
  'scallions',
  'soy',
  'oil',
  'seeds',
  'salt',
  'pepper',
  'oil',
  'egg',
  'pepper',
  'sugar',
  'water'],
 ['bread', 'milk', 'sugar', 'butter', 'salt', 'e

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

df = pd.DataFrame(mlb.fit_transform(new_ingredients),columns=mlb.classes_) #binary encode ingredients

In [12]:
df["target"] = classes
food_dict = df

In [13]:
#train
f = open(BASE_PATH + '/annotations/train_images.txt', "r")
train_images = f.read().split('\n')
f.close()
f = open(BASE_PATH + '/annotations/train_labels.txt', "r")
train_labels = f.read().split('\n')
f.close()

#val
f = open(BASE_PATH + '/annotations/val_images.txt', "r")
val_images = f.read().split('\n')
f.close()
f = open(BASE_PATH + '/annotations/val_labels.txt', "r")
val_labels = f.read().split('\n')
f.close()

#test
f = open(BASE_PATH + '/annotations/test_images.txt', "r")
test_images = f.read().split('\n')
f.close()
f = open(BASE_PATH + '/annotations/test_labels.txt', "r")
test_labels = f.read().split('\n')
f.close()

In [14]:
#train_images
len(train_labels)

68176

In [15]:
train_images = ["../input/food41/images/" + s + ".jpg" for s in train_images]
all_img_df = pd.DataFrame({'path': train_images, 'class_id': train_labels})
val_images = ["../input/food41/images/" + s + ".jpg" for s in val_images]
val_img_df = pd.DataFrame({'path': val_images, 'class_id': val_labels})
test_images = ["../input/food41/images/" + s + ".jpg" for s in test_images]
test_img_df = pd.DataFrame({'path': test_images, 'class_id': test_labels})

In [16]:
all_img_df = all_img_df[:-1]
val_img_df = val_img_df[:-1]
test_img_df = test_img_df[:-1]

In [17]:
all_img_df['class_name'] = all_img_df['path'].map(lambda x: os.path.split(os.path.dirname(x))[-1])
val_img_df['class_name'] = val_img_df['path'].map(lambda x: os.path.split(os.path.dirname(x))[-1])
test_img_df['class_name'] = test_img_df['path'].map(lambda x: os.path.split(os.path.dirname(x))[-1])

In [18]:
food_dict = food_dict.drop('', 1)

In [19]:
food_dict

,adobo,almond,anaheim chile,anchovy,anise,apple,arborio rice,asiago,avocado,baby arugula,baby back ribs,baby bok choy,bacon,baguette,baking,baklava,balsamic vinegar,basil,bay,bay leaf,beans,beef,beer,beets,berries,bird chile,biscuits,black pepper,bran,bread,brewed espresso,brie,broth,brown rice,brown sugar,buns,butter,cabbage,cajun,cake,...,seeds,serrano chilies,shallot,shiitake,shortening,shoulder roast,shrimp,sirloin,snails,soda,soy,soybean sprouts,spaghetti,spinach,spray,spring roll wrappers,steak,strawberries,sugar,sushi grade tuna,tahini,tartar,thyme,tofu,tomato,tortillas,turmeric,unflavored gelatin,vanilla,vanilla essence,vermouth,vinegar,water,wheat tortillas,whipping,white wine,worcestershire,yeast,zucchini,target
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,apple_pie
1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,baby_back_ribs
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,baklava
3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,beef_carpaccio
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,beef_tartare
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,beet_salad
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,beignets
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,bibimbap
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,bread_pudding
9,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,breakfast_burrito


In [20]:
#Dataframe for train images
new_data = []
for index, row in all_img_df.iterrows():
    #get binary encoding ingredients from lookup
    food = row["class_name"]
    path = row["path"]
    class_id = row["class_id"]
    binary_encod = food_dict.loc[food_dict["target"] == food]
    binary_encod["path"] = path
    binary_encod["class_id"] = class_id
    #print(binary_encod)
    #print((list(binary_encod.columns.values)))
    #print(len(np.array(binary_encod)[0]))
    new_data.append(np.array(binary_encod)[0])
    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [21]:
col_names = list(binary_encod.columns.values)
train_df = pd.DataFrame(new_data, columns = col_names)

In [22]:
#Dataframe for val images
val_data = []
for index, row in val_img_df.iterrows():
    #get binary encoding ingredients from lookup
    food = row["class_name"]
    path = row["path"]
    class_id = row["class_id"]
    binary_encod = food_dict.loc[food_dict["target"] == food]
    binary_encod["path"] = path
    binary_encod["class_id"] = int(class_id)
    val_data.append(np.array(binary_encod)[0])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [23]:
val_df = pd.DataFrame(val_data, columns = col_names)

In [24]:
#Dataframe for test images
test_data = []
for index, row in test_img_df.iterrows():
    #get binary encoding ingredients from lookup
    food = row["class_name"]
    path = row["path"]
    class_id = row["class_id"]
    binary_encod = food_dict.loc[food_dict["target"] == food]
    binary_encod["path"] = path
    binary_encod["class_id"] = int(class_id)
    test_data.append(np.array(binary_encod)[0])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [25]:
test_df = pd.DataFrame(test_data, columns = col_names)

In [26]:
train_df.to_hdf('train_df.h5','df',mode='w',format='table',data_columns=True)
val_df.to_hdf('val_df.h5','df',mode='w',format='table',data_columns=True)
test_df.to_hdf('test_df.h5','df',mode='w',format='table',data_columns=True)

/opt/conda/lib/python3.6/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'anaheim chile'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/opt/conda/lib/python3.6/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'arborio rice'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/opt/conda/lib/python3.6/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'baby arugula'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_

In [27]:

# train_df = pd.read_hdf("train_df.h5")
# test_df = pd.read_hdf("test_df.h5")

In [28]:
# train_df.shape[1]
# col_names
# ing_names = col_names[:-3]
# ing_names

In [29]:
# class DataWrapper(data.Dataset):
#     ''' Data wrapper for pytorch's data loader function '''
#     def __init__(self, image_df, resize):
#         self.dataset = image_df
#         self.resize = resize

#     def __getitem__(self, index):
#         c_row = self.dataset.iloc[index]
#         targets = ing_names + ['class_id']
#         image_path, target = c_row['path'], c_row[targets]  #image and target
#         #read as rgb image, resize and convert to range 0 to 1
#         image = cv2.imread(image_path, 1)
#         if self.resize:
#             image = cv2.resize(image, IMG_SIZE)/255.0 
#         else:
#             image = image/255.0
#         image = (torch.from_numpy(image.transpose(2,0,1))).float() #NxCxHxW
#         return image, int(target)

#     def __len__(self):
#         return self.dataset.shape[0] 

In [30]:
# train_dataset = DataWrapper(train_df, True)
# train_loader = torch.utils.data.DataLoader(train_dataset,shuffle=True, batch_size=batch_size, pin_memory=False)

# test_dataset = DataWrapper(test_df, True)
# test_loader = torch.utils.data.DataLoader(test_dataset,shuffle=True, batch_size=batch_size, pin_memory=False)

In [31]:
# model = models.resnet50(pretrained=True)
# #freeze all layers
# for param in model.parameters():
#      param.requires_grad = False
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)